In [58]:
import warnings
warnings.filterwarnings('ignore')


import tensorflow as tf
import numpy as np
import pickle as pkl
from sklearn import preprocessing as pre
#from pathos.pools import ProcessPool as Pool
tf.set_random_seed(1234)
np.random.seed(1234)
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"


In [25]:
#some hypers
GO='<START>'
STOP='<END>'
pad='<PAD>'
unknown='<UNKNOWN>'
BATCH=64
EPOCHS=50
embedding_size=512
lstm_units=1024
dropout_keep_prob=0.5


In [4]:
#load data
with open('./lingua.pkl') as f:
    trainCaptions=pkl.load(f)
trainCaptions=trainCaptions[:100000]

In [5]:
def processToken(caps):
    nk=[]
    
    for i in caps:
        t=[GO]+list(i)+[STOP]
        nk.append(t)
    return nk

In [6]:
trainCaptions=processToken(trainCaptions)


In [7]:
def listofwords(data):
    '''takes a list of sentences nd returns vocab'''
    a=[]
    for i in data:
        for j in i:
            if j not in a:
                a.append(j)
    return a

In [8]:
#load data
#with open('./vocab.pkl') as f:
#    trainVocab=pkl.load(f)

trainVocab=listofwords(trainCaptions)

In [9]:
#Label Encoder for output transform
pre_op=pre.LabelEncoder()
pre_op.fit(trainVocab)
onehoter=np.identity(len(pre_op.classes_),dtype=np.bool)


#word to int
from tqdm import tqdm
trainSeq=[]
for i in tqdm(range(len(trainCaptions))):
    trainSeq.append(pre_op.transform(trainCaptions[i]))

In [10]:
with open('./trainSeq.pkl','r+') as f:
    #pkl.dump(trainSeq,f)
    trainSeq=pkl.load(f)

In [12]:
trainSeqLen=[len(i) for i in trainSeq]

with open('./vocab.pkl','w+') as f:
    pkl.dump(obj=trainVocab,file=f)

#aeppending stops

MAX_LEN=max(trainSeqLen)
trainSeqReg=[np.pad(i,(0,MAX_LEN-len(i)),'constant',constant_values=pre_op.transform([STOP])) for i in tqdm(trainSeq)]
trainSeqReg=np.array(trainSeqReg)


In [13]:
with open('./trainSeqReg.pkl','r+') as f:
    #pkl.dump(trainSeqReg,f)
    trainSeqReg=pkl.load(f)

In [65]:
#Lets build the graph
tf.reset_default_graph()

In [66]:
source_seq = tf.placeholder(shape=(None,None),dtype=tf.int32)
source_seq_len = tf.placeholder(shape=(None,), dtype=tf.int32)
no_start_target_seq = tf.placeholder(shape=(None,None),dtype=tf.int32)
real_target_seq_len= tf.placeholder(shape=(None,), dtype=tf.int32)
batch_size = tf.placeholder(shape=(None),dtype=tf.int32)
keep_prob= tf.placeholder(dtype=tf.float32)

In [67]:
#output embeddings
embedding_matrix_encode = tf.get_variable(
    name="embedding_matrix_en",
    shape=[len(trainVocab), embedding_size],
    dtype=tf.float32)
decoder_input_embedded = tf.nn.embedding_lookup(embedding_matrix_encode, source_seq) 


#output embeddings
embedding_matrix_decode = tf.Variable(initial_value=tf.random_normal(shape=[trainVocabSize, embedding_size],dtype=tf.float32))
decoder_input_embedded = tf.nn.embedding_lookup(embedding_matrix_decode, target_seq) 


In [68]:
#encoder
encoderCell=tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(lstm_units),input_keep_prob=keep_prob,
                                          output_keep_prob=keep_prob)
#encoder_outputs,encoder_final_state=tf.nn.dynamic_rnn(cell=encoderCell,inputs=source_seq,sequence_length=source_seq_len,
#                dtype=tf.float32)



In [69]:
#Training helper and decoder
output_layer = tf.layers.Dense(len(trainVocab))
helper = tf.contrib.seq2seq.TrainingHelper(decoder_input_embedded,source_seq_len)
decoder = tf.contrib.seq2seq.BasicDecoder(encoderCell, helper,encoderCell.zero_state(BATCH,dtype=tf.float32),output_layer=output_layer)#,output_layer=projection_layer)
outputs, state, seq_len = tf.contrib.seq2seq.dynamic_decode(decoder)
logits = outputs.rnn_output


In [70]:
#loss and optimizer

#loss1
cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=no_start_target_seq,logits=logits)

target_weights = tf.sequence_mask(real_target_seq_len, source_seq_len[0], dtype=logits.dtype)

loss=tf.reduce_sum(cross_entropy*target_weights)

#train = tf.train.AdamOptimizer().minimize(total_loss)

#gradient clipping stackoverflow
optimizer = tf.train.AdamOptimizer()
gradients, variables = zip(*optimizer.compute_gradients(loss))
gradients, _ = tf.clip_by_global_norm(gradients, 10.0)
train = optimizer.apply_gradients(zip(gradients, variables))


In [71]:
#dont touch
maxtlen=max(trainSeqLen)
t_newlen=[maxtlen-1 for i in range(len(trainSeqLen))]

In [72]:
sess=tf.InteractiveSession()
tf.global_variables_initializer().run()

In [73]:
saver=tf.train.Saver()

In [74]:
#makes training batch
def getTrainBatch(indexs):
    targetBatch=np.array([trainSeqReg[i] for i in indexs])
    targetBatchLen=np.array([trainSeqLen[i] for i in indexs])
    return targetBatch,targetBatchLen



In [75]:
from tqdm import tqdm

#restore model
saver.restore(sess, "BestModel/model.ckpt")

In [76]:
#training starts here

print 'starting training'
training_losses=[]
valid_losses=[]
tData=np.arange(len(trainSeqReg))
for j in range(EPOCHS):
    np.random.shuffle(tData) #makes them iid
    training_loss=0
    for i in range(len(trainSeqReg)/BATCH):
        start=i*BATCH
        stop=(i+1)*BATCH
        targetBatch,targetBatchLen=getTrainBatch(tData[start:stop])
        _,lost=sess.run([train,loss],feed_dict={source_seq:targetBatch,
                                                
                                              source_seq_len:t_newlen[start:stop],
                                                real_target_seq_len:targetBatchLen,
                                                no_start_target_seq:np.array(targetBatch)[:,1:],
                                                batch_size:BATCH,keep_prob:dropout_keep_prob
                                                })
        
        training_loss+=lost
        #print lost,
    #calculate t_loss
    training_losses.append(training_loss/len(trainSeqReg))
    
    print "Epoch:%d training loss:%.4f"% (j,training_losses[-1])
    

starting training
Epoch:0 training loss:137.6477
Epoch:1 training loss:119.1827
Epoch:2 training loss:112.5506
Epoch:3 training loss:108.2925
Epoch:4 training loss:105.0517
Epoch:5 training loss:102.4475
Epoch:6 training loss:100.3081
Epoch:7 training loss:98.4834
Epoch:8 training loss:96.9632
Epoch:9 training loss:95.5894
Epoch:10 training loss:94.4401
Epoch:11 training loss:93.4733
Epoch:12 training loss:92.5448
Epoch:13 training loss:91.8041
Epoch:14 training loss:91.0990
Epoch:15 training loss:90.5031
Epoch:16 training loss:89.9369
Epoch:17 training loss:89.4489
Epoch:18 training loss:89.0159
Epoch:19 training loss:88.6044
Epoch:20 training loss:88.2419
Epoch:21 training loss:87.8993
Epoch:22 training loss:87.5914
Epoch:23 training loss:87.3346
Epoch:24 training loss:87.0739
Epoch:25 training loss:86.8331
Epoch:26 training loss:86.5965
Epoch:27 training loss:86.3903
Epoch:28 training loss:86.1751
Epoch:29 training loss:86.0033
Epoch:30 training loss:85.8338


KeyboardInterrupt: 

In [77]:
#save model
saver.save(sess, "model/LM.ckpt")

'model/LM.ckpt'

In [78]:
targetBatch,targetBatchLen=getTrainBatch(tData[0:64])
y=sess.run(outputs.sample_id,feed_dict={source_seq:targetBatch,
                                                
                                              source_seq_len:t_newlen[0:64],
                                                real_target_seq_len:targetBatchLen,
                                                no_start_target_seq:np.array(targetBatch)[:,1:],
                                                batch_size:1,keep_prob:1.0
                                                })


In [79]:
#processing summaries
gen_sum=y
summs=[]
for i in gen_sum:
    summ=''
    for j in i:
        
        if j!=58:
            summ = summ+' '+pre_op.inverse_transform(j)
    summs.append(summ[1:])

In [80]:
summs

['the a the question point i commission of the precautionary principle is to acceptable <END> the been said in the number of speakers <END> evening <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> 